In [1]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import editdistance
import itertools
# Enable inline plotting
%matplotlib inline

In [5]:
filename = 'namecleaned_big.csv'
df = pd.read_csv(filename, dtype=object)

In [6]:
df.head()

,Unnamed: 0,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp,contbr_nm_da
0,2,C00458844,P60006723,"Rubio, Marco","RUCINSKI, ROBERT",APO,AE,09096,US ARMY,PHYSICIAN,100,08-MAR-16,NaN,NaN,NaN,SA17A,1066300,SA17.1074981,P2016,"RUCINSKI, ROBERT"
1,3,C00458844,P60006723,"Rubio, Marco","DOHERTY, THERESA",APO,AE,09752,DODEA/DS,TEACHER,500,26-JUN-15,NaN,NaN,NaN,SA17A,1029436,SA17.769700,P2016,"DOHERTY, THERESA"
2,4,C00458844,P60006723,"Rubio, Marco","MARQUEZ, PANFILO",DPO,AE,09777,DEPARTMENT OF STATE,FOREIGN SERVICE OFFICER,100,07-MAR-16,NaN,NaN,NaN,SA17A,1066300,SA17.1074238,P2016,"MARQUEZ, PANFILO"
3,5,C00458844,P60006723,"Rubio, Marco","RECKER, ANGELA",APO,AE,00138,GOVERNMENT,ATTORNEY,388.25,18-MAR-16,NaN,NaN,DEBT RETIREMENT,SA17A,1066300,SA17.1153167,P2016,"RECKER, ANGELA"
4,6,C00458844,P60006723,"Rubio, Marco","RECKER, ANGELA",APO,AE,09012,GOVERNMENT,ATTORNEY,500,09-FEB-16,NaN,NaN,NaN,SA17A,1056862,SA17.1002064,P2016,"RECKER, ANGELA"


In [7]:
df.shape

(3801901, 20)

In [62]:
df_reduced = df[['contbr_st', 'cand_nm', 'contbr_nm_da', 'contbr_zip', 'contbr_employer', 'contb_receipt_amt', 'contb_receipt_dt']].copy()

In [63]:
df_reduced.shape

(3801901, 7)

In [64]:
df_reduced.head()

,contbr_st,cand_nm,contbr_nm_da,contbr_zip,contbr_employer,contb_receipt_amt,contb_receipt_dt
0,AE,"Rubio, Marco","RUCINSKI, ROBERT",09096,US ARMY,100,08-MAR-16
1,AE,"Rubio, Marco","DOHERTY, THERESA",09752,DODEA/DS,500,26-JUN-15
2,AE,"Rubio, Marco","MARQUEZ, PANFILO",09777,DEPARTMENT OF STATE,100,07-MAR-16
3,AE,"Rubio, Marco","RECKER, ANGELA",00138,GOVERNMENT,388.25,18-MAR-16
4,AE,"Rubio, Marco","RECKER, ANGELA",09012,GOVERNMENT,500,09-FEB-16


In [65]:
df_reduced = df_reduced[df_reduced['contb_receipt_amt'].astype('float64') > 0]
df_reduced = df_reduced.dropna(subset = ['contb_receipt_amt', 'contb_receipt_dt'])

In [69]:
df_reduced.to_csv('reduced_full_data.csv')

In [66]:
num_of_contr = df_reduced.groupby(['contbr_nm_da', 'contbr_zip'])['cand_nm'].nunique().sort_values().reset_index()
num_of_contr.head()

,contbr_nm_da,contbr_zip,cand_nm
0,"CONARE, PURNA RODMAN",02829,1
1,"O'CONNOR, MICHAEL",20007,1
2,"O'CONNOR, MICHAEL",29708,1
3,"O'CONNOR, MICHAEL",46219,1
4,"O'CONNOR, MICHAEL",94564,1


In [67]:
num_of_contr[num_of_contr['cand_nm'] >= 2].shape

(20524, 3)

In [68]:
df_reduced.groupby(['contbr_nm_da', 'contbr_zip'])['cand_nm'].count().shape

(685022,)

In [75]:
unq = lambda x: x.nunique()
df_reduced['num_of_candidates'] = df_reduced.groupby(['contbr_nm_da', 'contbr_zip'])['cand_nm'].transform(unq)
df_reduced.head()

,contbr_st,cand_nm,contbr_nm_da,contbr_zip,contbr_employer,contb_receipt_amt,contb_receipt_dt,num_of_candidates
0,AE,"Rubio, Marco","RUCINSKI, ROBERT",09096,US ARMY,100,08-MAR-16,1
1,AE,"Rubio, Marco","DOHERTY, THERESA",09752,DODEA/DS,500,26-JUN-15,1
2,AE,"Rubio, Marco","MARQUEZ, PANFILO",09777,DEPARTMENT OF STATE,100,07-MAR-16,1
3,AE,"Rubio, Marco","RECKER, ANGELA",00138,GOVERNMENT,388.25,18-MAR-16,1
4,AE,"Rubio, Marco","RECKER, ANGELA",09012,GOVERNMENT,500,09-FEB-16,1


In [91]:
df_donated_to_different = df_reduced[df_reduced['num_of_candidates'] > 1].copy()
df_donated_to_different.head()

,contbr_st,cand_nm,contbr_nm_da,contbr_zip,contbr_employer,contb_receipt_amt,contb_receipt_dt,num_of_candidates
42,AK,"Rubio, Marco","MELSON, CHARLENE",99503,RETIRED,50,27-FEB-16,2
44,AK,"Rubio, Marco","JANELLI, REBECCA T.",99507,RETIRED,100,16-JAN-16,3
45,AK,"Rubio, Marco","MITCHELL, WILLIAM W. MR.",99645,INFORMATION REQUESTED PER BEST EFFORTS,350,22-FEB-16,2
67,AK,"Rubio, Marco","WILLIAMS, DAVID L.",99709,RETIRED,50,03-FEB-16,5
68,AK,"Rubio, Marco","WILLIAMS, DAVID L.",99709,RETIRED,15,18-FEB-16,5


In [92]:
df_donated_to_different['contb_receipt_dt'] = pd.to_datetime(df_donated_to_different['contb_receipt_dt'])

In [100]:
df_donated_to_different = df_donated_to_different.sort_values(by='contb_receipt_dt')
df_donated_to_different.index.names = ['id']
df_donated_to_different.head()

,contbr_st,cand_nm,contbr_nm_da,contbr_zip,contbr_employer,contb_receipt_amt,contb_receipt_dt,num_of_candidates
id,,,,,,,,
58523,MI,"Rubio, Marco","FOSDICK, THEODORE D. MR.",48106,OMICRON INC.,100,2013-10-07,2
74817,NY,"Rubio, Marco","PALMER, EDITH P. MISS",10918,RETIRED,200,2013-10-11,3
58524,MI,"Rubio, Marco","FOSDICK, THEODORE D. MR.",48176,OMICRON INC.,200,2013-10-28,2
43905,GA,"Rubio, Marco","HARRIS, BOBBYE F. MS.",30701,RETIRED,100,2013-10-29,9
94908,TX,"Rubio, Marco","CAHOON, COLIN P. MR.",75229,SELF-EMPLOYED,400,2013-10-30,2


In [101]:
df_donated_to_different.to_csv('donated_to_different_guys.csv', index=True)